In [1]:
import csv
from tqdm import tqdm
import pandas as pd
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# import torch
# from transformers import AutoModel, AutoTokenizer
from vncorenlp import VnCoreNLP

In [ ]:
from pyspark import SparkContext, SparkConf
SparkContext.setSystemProperty('spark.executor.memory', '2g')
conf = SparkConf().setAppName("Process Comment").setMaster("spark://25.15.27.228:7077")
sc = SparkContext.getOrCreate(conf=conf)

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Process Comment').getOrCreate()

In [ ]:
class ProcssComment:
    def __init__(self,sc,spark,type = 'Comment'):
        self.sc = sc
        self.type = type
        self.spark = spark
        self.URI           = self.sc._gateway.jvm.java.net.URI
        self.Path          = self.sc._gateway.jvm.org.apache.hadoop.fs.Path
        self.FileSystem    = self.sc._gateway.jvm.org.apache.hadoop.fs.FileSystem
        self.Configuration = self.sc._gateway.jvm.org.apache.hadoop.conf.Configuration
        # self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # self.phobert = AutoModel.from_pretrained("vinai/phobert-base").to(self.device)
        # self.tokenizer_phobert = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
        self.rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
        self.getList()
        self.rates = ['1','2','3','4','5']
        self.dictRate = {}

    def getList(self):
        fs = self.FileSystem.get(self.URI("hdfs://cris:9000"), self.Configuration())
        status = fs.listStatus(self.Path(f'/shopee/{self.type}'))
        pathProducts = []
        for fileStatus in status:
            pathProducts.append(str(fileStatus.getPath()))
        self.path = pathProducts[1:]
    
    def processComment(self):
        for i in tqdm(range(len(self.path))):
            url = self.path[i]

            parDF2 = self.spark.read.parquet(url)
            cols = ['comment','rating_star']
            data = parDF2[cols].toPandas()
            for rate in self.rates:
                index = data[data['rating_star']==rate].index
                if index.shape[0] !=0:
                    if rate not in self.dictRate.keys():
                        self.dictRate[rate] = data.loc[index,:]
                    elif self.dictRate[rate].shape[0] < 50000:
                        self.dictRate[rate] = pd.concat([self.dictRate[rate],data.loc[index,:]],axis = 0)





            # data = self.getFeature(data,'comment')
            # return data
            # try:
            #     self.uploadHdfs(data)
            # except Exception as e:
            #     print(f'Error when upload to HDFS {e}')

    
    def getFeature(self,data,col):
        for x in tqdm(range(data.shape[0])):
            sentences = self.rdrsegmenter.tokenize(data[col].iloc[x])
            tmp = []
            for sentence in sentences:
                arr = " ".join(sentence)
                tmp.append(arr)
            data.loc[x,col] = ' '.join(tmp)

        return data
    


    def uploadHdfs(self,data):
        data = spark.createDataFrame(data.astype(str))
        data.coalesce(1).write.mode('append').parquet(f'hdfs://cris:9000/ProcessShopee/{self.type}/')
        print('Done upload HDFS')
    
    

In [ ]:
comment = ProcssComment(sc,spark)

In [ ]:
comment.processComment()